In [ ]:
from IPython.lib.deepreload import reload
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [ ]:
from analyze_phenotype import analyze_phenotype
from generate_great_file import generate_great_file
from misc import compute_contribution, compute_cos, compute_factor, build_from_guide, compute_contribution_gene, compute_contribution_cyto, compute_gene_contrib_to_cyto
from decomposition import decomposition

import numpy as np
import pathlib
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.gridspec as gs
import matplotlib.colors as mcolors
from matplotlib.colors import to_hex
from matplotlib import cm, colormaps

import random
import pathlib
from decimal import Decimal 

**Generate phenotype scores from GUIDE pipeline**

In [ ]:
pheno_folder = pathlib.Path("./all_phenos")
guide_scores = [str(path) for path in list(pheno_folder.rglob('*')) if '.tsv' in str(path) and 'degas' not in str(path)]

In [ ]:
gz = np.load('./../finngen_final/finngen_ld_0_3_decomp_bl_ll.npz', allow_pickle=True)
cos_guide = gz['cos_phe']

pheno_guide = gz['label_phe']
factor_guide = gz['factor_phe']         # measure pheno contrib per latent
contrib_guide = gz['contribution_phe']  # measure latent contribution per pheno

contrib_var_guide = gz['contribution_var']
var_labels = gz['label_var']

print(len(pheno_guide))
print(contrib_guide.shape)

In [ ]:
contrib_phenos = np.load('./../finngen_final/finngen_ld_0_3_decomp_bl_ll.npz', allow_pickle=True)['contribution_phe']
phenos = np.load('./../finngen_final/finngen_ld_0_3_decomp_bl_ll.npz', allow_pickle=True)['label_phe']

print(contrib_phenos.shape)

ranked = []
for i, col in enumerate(contrib_phenos.T):
    ranked.append((i, sum(sorted(col, reverse=True)[:3])))
#for p in phenos:

ranked =sorted(ranked, key=lambda x: x[1], reverse=True)
interested = [31]

for l, sc in ranked:
    assoc = [(i, score) for i, score in enumerate(contrib_phenos.T[l])]
    pinds = sorted(assoc, key=lambda x: x[1], reverse=True)[:10]
    #print(pinds)
    print('Lat', l, sc)

    for i,s in pinds:
        print(phenos[i],s)
    print('\n\n')


In [ ]:
cmap = plt.get_cmap('rainbow')
print(len(pheno_guide))
colors = cmap(np.linspace(0, 1.0, len(pheno_guide)))
print(colors)

colors = list(map(lambda x: to_hex(x), colors))
contrib_guide.shape

In [ ]:
def gen_barplot_data(pheno_labels, scores, is_guide, colors):
    if is_guide:
        method_str = "GUIDE"
    else:
        method_str = "TSVD"
    ls_dict = dict()
    lat_labels = [f"{method_str} Lat{i}" for i in range(100)]
    for label, row in zip(lat_labels, scores.T):
        ls_dict[label] = row
        assert np.sum(row) - 1.0 < 1e-6, "Likely not valid pheno contributions to latents."
    #print(ls_dict)
    
    df = pd.DataFrame(data=ls_dict, index=pheno_labels)
    df.index.name = 'phenotype'
    try:
        df['color'] = colors
    except:
        pass
    return df
        

guide_df = gen_barplot_data(pheno_guide, contrib_guide, True, colors)
degas_df = gen_barplot_data(pheno_guide, contrib_degas, False, colors)

**Function for generating stacked bar plots (trait contribution to latent factor)**

In [ ]:
def plot_stacked_bar_per_pheno_combined(df, cols, figname=None, is_guide=None):
    if is_guide:
        title='Contributions of top three traits to GUIDE latent factors'
    else:
        title='Contributions of top three traits to TSVD latent factors'
    plt.clf()
    to_plot = {'top contributing trait':[],
              '2nd highest contributor':[],
              '3rd highest contributor':[],
              'other':[]}
    
    color_map = {'top contributing trait' : '#78a3d4',
                 '2nd highest contributor' : '#7ec4cf',
                 '3rd highest contributor' : '#9cadce',
                 'other' : 'lightgrey'}
    
    
    for lat in cols:
        data = df[lat].sort_values(ascending=False)[:3]
        to_plot['top contributing trait'].append(data.iloc[0])
        to_plot['2nd highest contributor'].append(data.iloc[1])
        to_plot['3rd highest contributor'].append(data.iloc[2])
        to_plot['other'].append(1.0-data.iloc[0]-data.iloc[1]-data.iloc[2])
        
    to_plot = pd.DataFrame(to_plot, index=cols)
    to_plot.plot.bar(stacked=True, color=color_map).legend(loc='center left', bbox_to_anchor=(1, 0.5))
    #plt.xticks(rotation=45)
    plt.ylabel('Phenotype Contribution Scores')
    plt.xlabel('Latent Label')
    plt.title(title)
    
    if figname:
        plt.savefig(figname, dpi=200, bbox_inches="tight")
    plt.show()
        
def retain_permutation(l1, l2):
    lst = [None for _ in range(len(l2))]
    for i, (x, y) in enumerate(zip(l1, l2)):
        lst[i] = (i, x, y)
        
    lst = sorted(lst, key=lambda x: int(x[1].split('GUIDE Lat')[1]))
    return [ele for _, ele, _ in lst], [ele for _, _, ele in lst]
    

# use this to maintain lists of latent factors to investigate
interested_traits_guide= [
                    'GUIDE Lat72',
                    'GUIDE Lat10',
                    'GUIDE Lat93',
                    'GUIDE Lat18',
                    'GUIDE Lat69',
                    'GUIDE Lat21',
                    'GUIDE Lat53',
                    'GUIDE Lat9',
                    'GUIDE Lat58',
                    'GUIDE Lat61',
                    'GUIDE Lat31',
                    'GUIDE Lat15',
                    'GUIDE Lat6',
                    'GUIDE Lat25',
                    'GUIDE Lat88']

interested_traits_deg = [
                    'TSVD Lat71',
                    'TSVD Lat21',
                    'TSVD Lat38',
                    'TSVD Lat27',
                    'TSVD Lat33',
                    'TSVD Lat11',
                    'TSVD Lat2',
                    'TSVD Lat28',
                    'TSVD Lat40',
                    'TSVD Lat52',
                    'TSVD Lat39',
                    'TSVD Lat41',
                    'TSVD Lat39',
                    'TSVD Lat2',
                    'TSVD Lat22']

# generate the bar plots
interested_traits_guide, interested_traits_deg = retain_permutation(interested_traits_guide, interested_traits_deg)
plot_stacked_bar_per_pheno_combined(guide_df, interested_traits_guide, figname='guide_contrib_general.png', is_guide=True)
plot_stacked_bar_per_pheno_combined(degas_df, interested_traits_deg, figname='degas_contrib_general.png', is_guide=False)

In [ ]:
def plot_stacked_bar_per_pheno_detailed(df, cols, spacing=2.3, figname=None):
    plt.clf()
    plt.rcParams['figure.constrained_layout.use'] = False
    nplots = len(cols)

    fig, axes = plt.subplots(1, nplots, figsize=(20, 10))
    colors= ['#78a3d4', '#7ec4cf', '#9cadce', 'lightgrey']


    for i, lat in enumerate(cols):
        data = df[lat].sort_values(ascending=False)[:10]
        phenos = list(data.index)
        to_plot = [data.iloc[0], data.iloc[1], data.iloc[2], 1.0-data.iloc[0]-data.iloc[1]-data.iloc[2]]
        bar_width=0.1
        plt.sca(axes[i])
        axes[i].bar(lat, to_plot[0], color=colors[0], label=phenos[0], width=bar_width)
        axes[i].bar(lat, to_plot[1], bottom=to_plot[0], color=colors[1], label=phenos[1], width=bar_width)
        axes[i].bar(lat, to_plot[2], bottom=to_plot[1]+to_plot[0], color=colors[2], label=phenos[2], width=bar_width)
        
        axes[i].bar(lat, to_plot[3], bottom=to_plot[2]+to_plot[1]+to_plot[0], color=colors[3], label='other', width=bar_width)
        
        
        axes[i].legend(loc='center left', bbox_to_anchor=(1, 0.7, 1., 0.), fontsize="10", )
        if i == 0:
            plt.ylabel('Phenotype Contribution Scores')
        elif i == 2:
            axes[i].set_yticklabels([])
        else:
            axes[i].set_yticklabels([])
            
    plt.subplots_adjust(wspace=spacing)
    
    if figname:
        print(figname)
        plt.savefig(figname, dpi=200 , bbox_inches = "tight")
    
    print(axes)
    
        
def retain_permutation(l1, l2):
    lst = [None for _ in range(len(l2))]
    for i, (x, y) in enumerate(zip(l1, l2)):
        lst[i] = (i, x, y)
        
    lst = sorted(lst, key=lambda x: int(x[1].split('GUIDE Lat')[1]))
    return [ele for _, ele, _ in lst], [ele for _, _, ele in lst]
interested_traits_guide = [
                    'GUIDE Lat80', 
                    'GUIDE Lat27', 
                    'GUIDE Lat23', 
]

interested_traits_deg = [
                    'TSVD Lat64', 
                    'TSVD Lat0', 
                    'TSVD Lat88', 
                    ]

interested_traits_guide.extend(interested_traits_deg)
merged_df = pd.concat([guide_df, degas_df], axis=1)
plot_stacked_bar_per_pheno_detailed(merged_df, interested_traits_guide, spacing=8, figname='both_cigarettes.png')#figname='degas_lat_contribs.png')

In [ ]:
label_var = np.load('./dev_allNonMHC_z_center_p0001_100PCs_20180129.npz', allow_pickle=True)['label_var']

In [ ]:
def plot_stacked_bar_per_pheno_detailed_latent_loadings(df, cols, phenames, spacing=2.3, figname=None, topn=3, read_colors=True, write_colors=False):
    plt.clf()
    plt.rcParams['figure.constrained_layout.use'] = False
    nplots = len(cols)

    fig, axes = plt.subplots(1, nplots, figsize=(20, 10))
    #print(axes)
    
    if read_colors:
        with open('colors.txt', 'r') as f:
            colors = []
            for l in f.read().split('\n'):
                try:
                    color = l.strip('()')
                    color = color.split(',')
                    color = list(map(lambda x: float(x.strip()), color))
                    colors.append(mcolors.to_rgb(color))
                except:
                    colors.append(color)
        colors[topn] = 'lightgrey'

    else:
        color_set = list(map(cm.get_cmap('Blues', topn), [1.0/topn * i for i in range(topn)]))
        colors = []
        
        random.Random(45).shuffle(color_set)
        colors = color_set
        colors.append('lightgrey')

    if write_colors:
        with open('colors.txt', 'w') as f:
            s = ''
            for c in colors:
                s+=(f'{c}\n')
            f.write(s[:-1])

    pcstrings=['PCs1', 'PCs2', 'PCs3', 'PCs4']
    for i, lat in enumerate(cols):
        data = df[lat].sort_values(ascending=False)[:topn]
        phenos = phenames
        to_plot = []
        ss = 0
        for c in range(topn):
            curr = data.iloc[c]
            ss += curr
            to_plot.append(curr)
        to_plot.append(1-ss)

        bar_width=0.1
        plt.sca(axes[i])
        cs = 0
        for c in range(topn):
            
            axes[i].bar(phenos[i], to_plot[c], bottom=cs, color=colors[c], label=df[pcstrings[i]][c], width=bar_width)
            cs += to_plot[c]
            

        axes[i].bar(phenos[i], to_plot[topn], bottom=cs, color=colors[topn], label='other', width=bar_width)
        plt.ylim([0, 1])

        
        
        axes[i].legend(loc='center left', bbox_to_anchor=(1.05, .5, .5, 0.), fontsize="10", )
        if i == 0:
            plt.ylabel(f'Latent Contribution Scores')
        else:
            axes[i].set_yticklabels([])
    
    plt.subplots_adjust(wspace=spacing)
    
    if figname:
        print(figname)
        plt.savefig(figname, dpi=200 , bbox_inches = "tight")
        plt.show()
    
def retain_permutation(l1, l2):
    lst = [None for _ in range(len(l2))]
    for i, (x, y) in enumerate(zip(l1, l2)):
        lst[i] = (i, x, y)
        
    lst = sorted(lst, key=lambda x: int(x[1].split('GUIDE Lat')[1]))
    return [ele for _, ele, _ in lst], [ele for _, _, ele in lst]
    

interested_traits_guide = [
                    'GUIDE Lat72', 
                    'GUIDE Lat10',
                    ]

interested_traits_deg = [
                    'TSVD Lat71', 
                    'TSVD Lat64',
                    'TSVD Lat21', 
                    'TSVD Lat19',
                    ]

fev_df = pd.read_csv('./all_phenos/phenotypes/guide/Arm_fat_percentage__left_/squared_cosine_scores.tsv', sep='\t')
fev2_df = pd.read_csv('./all_phenos/phenotypes/guide/Arm_fat_percentage__right_/squared_cosine_scores.tsv', sep='\t')
fev3_df = pd.read_csv('./all_phenos/phenotypes/guide/Arms_tissue_fat_percentage/squared_cosine_scores.tsv', sep='\t')
fev4_df = pd.read_csv('./all_phenos/phenotypes/guide/Diabetes_diagnosed_by_doctor/squared_cosine_scores.tsv', sep='\t')

colname = ["css1", "css2", "css3", "css4"]
merged = pd.concat([fev_df, fev2_df, fev3_df, fev4_df], axis=1,)
merged.columns = ['Rank1', 'PCs1', 'css1', 'Rank2', 'PCs2', 'css2', 'Rank3', 'PCs3', 'css3', 'Rank4', 'PCs4', 'css4']
print(merged)
plot_stacked_bar_per_pheno_detailed_latent_loadings(merged, colname, 
                                                    ['Arm Fat Percentage (Left)','Arm Fat Percentage (Right)', 'Arm Fat Percentage', 'Diabetes Diagnosed by Doctor'], 
                                                    read_colors=True,
                                                    write_colors=False,
                                                    spacing=1.7, figname='fevstacked.png', topn=10)

In [ ]:
contribution_var_guide = np.load('./guide_all_100lat_bl_ll.npz', allow_pickle=True)['contribution_var']
contribution_var_degas = np.load('./dev_allNonMHC_z_center_p0001_100PCs_20180129.npz', allow_pickle=True)['contribution_var']

var_gene_file='./snp_gene_pq.txt'

var_labels = dz['label_var']

gene_c_guide, names_guide = compute_contribution_gene(label_var, contribution_var_guide, var_gene_file)
gene_c_degas, names_degas = compute_contribution_gene(label_var, contribution_var_degas, var_gene_file)


gcg = dict(zip([f'GUIDE Lat{x}' for x in range(100)], gene_c_guide.T))
gcd = dict(zip([f'TSVD Lat{x}' for x in range(100)], gene_c_degas.T))

gene_contrib_guide = pd.DataFrame(data=gcg, index=names_guide)
gene_contrib_degas = pd.DataFrame(data=gcd, index=names_degas)
all_degas = np.load('./dev_allNonMHC_z_center_p0001_100PCs_20180129.npz', allow_pickle=True)

In [ ]:
gonet_guide = gene_contrib_guide['GUIDE Lat10'].nlargest(15000)
gonet_guide.to_csv('gonet_lat72_guide.csv', sep='\t', header=None)

In [ ]:
gc_deg = gene_contrib_degas.to_numpy()
gc_gde = gene_contrib_guide.to_numpy()

pc_deg = degas_df.to_numpy()
pc_gde = guide_df.to_numpy()

guide_names = [x for x in range(100)]
degas_names = [x for x in range(100)]

deg_scores = []
for name, gc, pc in zip(degas_names, gc_deg.T, pc_deg.T):
    gc = np.sum(np.sort(gc)[::-1][:40])
    pc = np.sum(np.sort(pc)[::-1][:3])
    deg_scores.append((name, pc + gc))
    
guide_scores = []
for name, gc, pc in zip(guide_names, gc_gde.T, pc_gde.T):
    gc = np.sum(np.sort(gc)[::-1][:20])*3
    pc = np.sum(np.sort(pc)[::-1][:3]) 
    guide_scores.append((name, pc + gc))
guide_scores = sorted(guide_scores, key=lambda x: x[1], reverse=True)


In [ ]:
def get_safe_phe_name(phe):
    return ''.join([c if (c.isalnum() or c in ['_', '.'] ) else '_' for c in phe])    

pheno_labels = gz['label_phe']
guide_interested = list(map(lambda x: x[0], guide_scores[:20]))
guide_phenos_good = []
for ind in guide_interested:
    print(contrib_guide[:, ind].shape)
    px = np.argmax(contrib_guide[:, ind])
    guide_phenos_good.append(pheno_labels[px])

degas_interested = []
for gp in guide_phenos_good:
    df = pd.read_csv(f'./all_phenos/phenotypes/degas/{get_safe_phe_name(gp)}/squared_cosine_scores.tsv', sep='\t', header=0)
    i = 0
    curr = list(df['PC_(zero_based)'])[i]
    while curr in degas_interested:
        curr = list(df['PC_(zero_based)'])[i]
        i+=1
    degas_interested.append(list(df['PC_(zero_based)'])[0])
    
guide_interested = list(map(lambda x: f'GUIDE Lat{x}', guide_interested))
degas_interested = list(map(lambda x: f'TSVD Lat{x}', degas_interested))

guide_interested, degas_interested = retain_permutation(guide_interested, degas_interested)

In [ ]:
plot_stacked_bar_per_pheno_detailed(gene_contrib_degas, degas_interested, spacing=1., figname='general_gene_degas.png', topn=40, is_guide=False, legend=False, general=True)
plot_stacked_bar_per_pheno_detailed(gene_contrib_guide, guide_interested, spacing=1., figname='general_gene_guide.png', topn=40, is_guide=True, legend=False, general=True)
plot_stacked_bar_per_pheno_combined(guide_df, guide_interested, figname='guide_contrib_general.png', is_guide=True)
plot_stacked_bar_per_pheno_combined(degas_df, degas_interested, figname='degas_contrib_general.png', is_guide=False)

In [ ]:
var_gene_file='../finngen_final/finngen_ld_0_3_s2g.txt'

gz = np.load('../finngen_final/finngen_ld_0_3_decomp_bl_ll.npz', allow_pickle=False)
var_labels = gz['label_var']
#var_labels = list(map(lambda x: x.replace(':', '-'), var_labels))
contribution_var_guide = np.load('../finngen_final/finngen_ld_0_3_decomp_bl_ll.npz', allow_pickle=True)['contribution_var']
cyto_c_guide, names_guide, gene2cyto_dict_guide = compute_gene_contrib_to_cyto(var_labels, 
                                                                               contribution_var_guide, 
                                                                               var_gene_file)

ccg = dict(zip([f'GUIDE Lat{x}' for x in range(100)], cyto_c_guide.T))
cyto_contrib_guide = pd.DataFrame(data=ccg, index=names_guide)


In [ ]:
def plot_stacked_bar_per_pheno_detailed_cyto(df, cols, spacing=2.3, path_to_rel_df=None, figname=None, topn=3, topn_genes=4, colors=None, gene2cyto_dict=None):
    if not gene2cyto_dict:
        print('You need a gene to cyto (pq nomenclature) dictionary.')
        return
        
    plt.clf()
    plt.rcParams['figure.constrained_layout.use'] = False
    nplots = len(cols)

    fig, axes = plt.subplots(1, nplots, figsize=(20, 10))
    
    if not colors:
        color_set = list(map(cm.get_cmap('Blues', topn), [1.0/topn * i for i in range(topn)]))
        colors = []
        random.Random(45).shuffle(color_set)
        color_set[0], color_set[1] = color_set[1], color_set[0]
        colors = color_set
        #print(colors)
        colors[4] = np.asarray([174, 198, 207])/255
        colors.append('lightgrey')
        
    elif colors:
        colors = colors
        colors[4], colors[8] = colors[8], colors[4]
        colors[topn] = 'lightgrey'
        

    genes = list(df.index)
    latent_info = dict()
    
    middle_ys_all = []
    contribs = []
    for path in path_to_rel_df:
        contrib_df = pd.read_csv(path, 
                                 sep='\t',
                                 header=0,
                                 index_col=1) 
    contrib_strings = ['' for _ in range(len(cols))]
    for i, lat in enumerate(cols):
        for cont in contribs:
            if cont[i] > 0.001:
                contrib_strings[i] += f"{cont[i]:.2f}%\n"
            else:
                contrib_strings[i] += f"{Decimal(str(cont[i])):.2E}%\n"
            
    for i, lat in enumerate(cols):
        cyto_map = dict()
        for g, cont in zip(df.index, list(df[lat])):
            curr_cyto = gene2cyto_dict[g]
            try:
                cyto_map[curr_cyto][0] += cont
                cyto_map[curr_cyto][1].append(g)
                cyto_map[curr_cyto][2].append(cont)
            except:
                cyto_map[curr_cyto] = [0, [g], [cont]]
            
        cyto_info = []
        for cyto, (total_cont, genes, ind_cont) in cyto_map.items():
            gene_info_sorted = sorted(zip(genes, ind_cont), key=lambda x: x[1], reverse=True)
            cyto_info.append([cyto, total_cont, gene_info_sorted])
        cyto_info = sorted(cyto_info, key=lambda x: x[1], reverse=True)
        to_plot = []
        ss = 0
        
        for c in range(topn):
            curr = cyto_info[c][1]
            ss += curr
            to_plot.append(curr)
        
        to_plot.append(1-ss)
        bar_width=0.1
        plt.sca(axes[i])
        cs = 0
        
        middle_ys = []
        for c in range(topn):
            if c >= topn:
                continue
            label = f'{cyto_info[c][0]} ({(100*cyto_info[c][1]):.2f}%)\n'
            n_gene = 0
            curr_ind = 0
            #print(c)
            while n_gene < topn_genes:
                if cyto_info[c][2][curr_ind][0] != "NONE":
                    gene_name, g_cont = cyto_info[c][2][curr_ind][0], cyto_info[c][2][curr_ind][1]
                    label += f' - {gene_name}\n'
                    n_gene+=1
                    curr_ind += 1
                else:
                    curr_ind += 1
                    gene_name, g_cont = cyto_info[c][2][curr_ind][0], cyto_info[c][2][curr_ind][1]
                    label += f' - {gene_name}\n'
                    curr_ind += 1
                    n_gene+=1
            label = label[:-1]
            # USE THIS TO TURN CONTRIB STRINGS OFF
            #axes[i].bar(lat+'\n'+contrib_strings[i], to_plot[c], bottom=cs, color=colors[c], label=label, width=bar_width)
            axes[i].bar(lat, to_plot[c], bottom=cs, color=colors[c], label=label, width=bar_width)

            cs += to_plot[c]

        middle_ys.append(cs + to_plot[topn]/2)
        middle_ys_all.append(middle_ys)

        # USE THIS FOR TURNING CONTRIB STRINGS OFF
        #axes[i].bar(lat+'\n'+contrib_strings[i], to_plot[topn], bottom=cs, color=colors[topn], label=f'other ({100*(1-cs):.2f}%)', width=bar_width)
        axes[i].bar(lat, to_plot[topn], bottom=cs, color=colors[topn], label=f'other ({100*(1-cs):.2f}%)', width=bar_width)
        plt.ylim([0, 1])

        # invert the comments here to reverse the order of the colors/labels
        #axes[i].legend(loc='center left', bbox_to_anchor=(1.05, .5, .5, 0.), fontsize="10", )
        handles, labels = axes[i].get_legend_handles_labels()
        leg = axes[i].legend(handles[::-1], labels[::-1], loc='lower left', bbox_to_anchor=(1.05, 0.0, .5, 0), fontsize="10", )
        leg.get_frame().set_linewidth(0.0)

        if i == 0:
            plt.ylabel(f'Variant Contribution Scores', fontsize=18)
        else:
            axes[i].set_yticklabels([])
                
    plt.subplots_adjust(wspace=spacing)
    
    if figname:
        print(figname)
        plt.savefig(figname, dpi=200 , bbox_inches = "tight")
        plt.show()
        
    return middle_ys_all, colors[:topn] + ['lightgrey']

interested_traits_guide = [
                    'GUIDE Lat10', 
                    'GUIDE Lat27', 
                    'GUIDE Lat23', 
]

interested_traits_guide = [
                    'GUIDE Lat57', 
                    'GUIDE Lat62', 
                    'GUIDE Lat39', 
                    'GUIDE Lat6', 
                    'GUIDE Lat14', 
]

interested_traits_guide = [
                    'GUIDE Lat78', 
                    'GUIDE Lat70',   
]
interested_traits_guide = ['GUIDE Lat31', 'GUIDE Lat88', 'GUIDE Lat68', 'GUIDE Lat64', 'GUIDE Lat48']


merged_df = pd.concat([cyto_contrib_guide, ], axis=1)

colors = []
with open('./colors.txt', 'r') as f:
    for c in f.read().split('\n'):
        try:
            colors.append(list(map(lambda x: float(x), [x.strip(' ()') for x in c.split(',')])))
        except:
            colors.append([x.strip(' ()') for x in c.split(',')])

paths = [
    './finngen/phenotypes/guide/Respiratory_distress_of_newborn/squared_cosine_scores.tsv',
    './finngen/phenotypes/guide/ILD_related_to_systemic_autoimmune_disease/squared_cosine_scores.tsv',
    './finngen/phenotypes/guide/ILD_related_respiratory_insufficiency/squared_cosine_scores.tsv',
    './finngen/phenotypes/guide/Respiratory_and_cardiovascular_disorders_specific_to_the_perinatal_period/squared_cosine_scores.tsv',
    './finngen/phenotypes/guide/Respiratory_disorders_in_diseases_classified_elsewhere/squared_cosine_scores.tsv',
    './finngen/phenotypes/guide/Polyarteritis_with_lung_involvement__Churg_Strauss__EGPA/squared_cosine_scores.tsv',
    './finngen/phenotypes/guide/Pneumoconiosis_due_to_other_inorganic_dusts/squared_cosine_scores.tsv'
]
         
middle_ys_cyto, bar_colors = plot_stacked_bar_per_pheno_detailed_cyto(merged_df, 
                                                                      interested_traits_guide, 
                                                                      spacing=3.5,
                                                                      path_to_rel_df=paths,
                                         figname='combined_contribs_cyto_finngen_340k.png', 
                                         topn=12, 
                                         topn_genes=2,
                                         colors=colors, 
                                         gene2cyto_dict=gene2cyto_dict_guide)
print(gz['contribution_phe'].shape)

In [ ]:
contrib_df = pd.read_csv('all_phenos/phenotypes/guide/Forced_expiratory_volume_in_1_second__FEV1___Best_measure/squared_cosine_scores.tsv', 
                             sep='\t',
                             header=0,
                             index_col=1)

**Generates phenotype contribution to latent factors**

In [ ]:
gz = np.load('finngen_ld_pruned_archive.npz', allow_pickle=True)

def plot_stacked_bar_per_pheno_detailed(df, cols, topn=3, cust_cmap=None, spacing=2.3, figname=None):
    plt.clf()
    plt.rcParams["axes.prop_cycle"] = plt.cycler("color", plt.cm.tab20.colors)
    plt.rcParams['figure.constrained_layout.use'] = False
    nplots = len(cols)

    colors = list(pd.read_csv('./colors.txt', header=None, sep='\t').iloc[:-2][0])
    colors = list(map(lambda x: x.strip('()').split(','), colors))
    colors = list(map(lambda x: list(map(lambda y: float(y.strip()), x)), colors))
    colors = list(map(lambda x: to_hex((x[0], x[1], x[2])), colors))
    to_hex((0.2,0.2,0.9))

    pheno_color_dict = {}
    top3_per_lat = dict()
    phenos_seen = []
    color_i = 0

    for i, lat in enumerate(cols):
        data = df[lat].sort_values(ascending=False)
        phenos = list(data.index)
        top3_per_lat[lat] = phenos[:topn]
        for p in phenos[:topn]:
            if p not in phenos_seen:
                pheno_color_dict[p] = colors[color_i]
                color_i += 1
                phenos_seen.append(p)
        
    num_phenos = len(phenos_seen)
    num_lats = len(cols)
    print('topn per lat', top3_per_lat)

    phe_contrib_mat = np.zeros((num_lats, num_phenos))
    for i, lat in enumerate(cols):
        for j, pheno in enumerate(phenos_seen):
            if pheno in top3_per_lat[lat]:
                try:
                    phe_contrib_mat[i, j] = df[lat][pheno]
                except:
                    phe_contrib_mat[i, j] = df[lat][pheno][1]
            else:
                phe_contrib_mat[i, j] = 0
                        
    width = 0.35       # the width of the bars: can also be len(x) sequence

    ind = np.arange(num_lats)

    cumulative_vec = np.zeros((num_lats))

    if cust_cmap:
        cmap = dict(zip(phenos_seen, cust_cmap))
        cmap['other'] = 'lightgrey'
    else:
        cmap = colormaps['tab20']

    print(phe_contrib_mat.shape)
    for i, lat in enumerate(cols):
        to_plot = list(zip(list(phe_contrib_mat[i, :]), phenos_seen))
        for j, pheno in enumerate(phenos_seen):
            plt.bar(i, to_plot[j][0], width, bottom=cumulative_vec[i],color=cmap[to_plot[j][1]], label=to_plot[j][1])
            cumulative_vec[i] = cumulative_vec[i] + to_plot[j][0]
        
    plt.bar(ind, 1 - np.sum(phe_contrib_mat, axis=1), width, 
            bottom=cumulative_vec, 
            label='other', 
            color='lightgrey',)
    
    phenos_seen.append('other')
    leg = plt.legend(phenos_seen, fontsize=10, bbox_to_anchor=(1., 1.05), )
    leg.legendHandles[-1].set_color('lightgrey')

    plt.xticks(ind, cols, fontsize=10, rotation=35)
    plt.xlabel('')
    plt.ylabel('Phenotype Contribution Scores', fontsize=10)
    if figname:
        print(figname)
        plt.savefig(figname, dpi=200 , bbox_inches = "tight")
        plt.show()
        
def retain_permutation(l1, l2):
    lst = [None for _ in range(len(l2))]
    for i, (x, y) in enumerate(zip(l1, l2)):
        lst[i] = (i, x, y)
        
    lst = sorted(lst, key=lambda x: int(x[1].split('GUIDE Lat')[1]))
    return [ele for _, ele, _ in lst], [ele for _, _, ele in lst]

interested_traits_deg = [
                    'TSVD Lat2', 
                    'TSVD Lat0',
                    #'TSVD Lat21', 
                    #'TSVD Lat19',
                    #'TSVD Lat9',
                    #'TSVD Lat53'
                    ]

interested_traits_guide = [
                    'GUIDE Lat70',
                    'GUIDE Lat94',
                    'GUIDE Lat9',
                    'GUIDE Lat15',
                    'GUIDE Lat29'
]

contrib_guide = gz['contribution_phe']  # measure latent contribution per pheno
pheno_guide = gz['label_phe']
print(contrib_guide.shape)
print(pheno_guide.shape)
guide_df = gen_barplot_data(pheno_guide, contrib_guide, True, colors)
merged_df = pd.concat([guide_df], axis=1)

# custom map to keep similar phenos the same color
manuscript_colors = ['steelblue', 'lightseagreen', 'aqua', 'cornflowerblue', 'royalblue', 'deepskyblue', 'mediumturquoise',
           'darkorange',
           'olivedrab', 'yellowgreen',
           'violet', 'purple',
           'lightpink',
           'goldenrod', 'darkgoldenrod',
           'red', 'maroon', 'grey']

plot_stacked_bar_per_pheno_detailed(guide_df, interested_traits_guide, 
                                    cust_cmap=manuscript_colors,
                                    spacing=8, 
                                    topn=5,
                                    figname='both_cigarettes.png')#figname='degas_lat_contribs.png')

In [ ]:
def pheno_contribs_to_lat(df, cols, topn=3, spacing=2.3, figname=None):
    plt.clf()
    fig = plt.figure()
    plt.rcParams["axes.prop_cycle"] = plt.cycler("color", plt.cm.tab20.colors)
    plt.rcParams['figure.constrained_layout.use'] = False
    nplots = len(cols)

    top3_per_lat = dict()
    phenos_seen = []

    for i, lat in enumerate(cols):
        data = df[lat].sort_values(ascending=False)
        phenos = list(data.index)
        top3_per_lat[lat] = phenos[:topn]
        for p in phenos[:topn]:
            if p not in phenos_seen:
                phenos_seen.append(p)

    num_phenos = len(phenos_seen)
    num_lats = len(cols)
    phe_contrib_mat = np.zeros((num_lats, num_phenos))
    for i, lat in enumerate(cols):
        for j, pheno in enumerate(phenos_seen):
            if pheno in top3_per_lat[lat]:
                try:
                    phe_contrib_mat[i, j] = df[lat][pheno]
                except:
                    phe_contrib_mat[i, j] = df[lat][pheno][1]
            else:
                phe_contrib_mat[i, j] = 0
                        
    width = 0.35       # the width of the bars: can also be len(x) sequence

    ind = np.arange(num_lats)

    cumulative_vec = np.zeros((num_lats))

    print(phe_contrib_mat.shape)
    for i, lat in enumerate(cols):
        to_plot = list(zip(list(phe_contrib_mat[i, :]), phenos_seen))
        for j, pheno in enumerate(phenos_seen):
            print(to_plot[j][0], to_plot[j][1])
            plt.bar(i, to_plot[j][0], width, bottom=cumulative_vec[i], label=to_plot[j][1])
            cumulative_vec[i] = cumulative_vec[i] + to_plot[j][0]
        
    plt.bar(ind, 1 - np.sum(phe_contrib_mat, axis=1), width, bottom=cumulative_vec, label='other', color='lightgrey',)
    phenos_seen.append('other')
    leg = plt.legend(phenos_seen, fontsize=8, bbox_to_anchor=(1., 1.05), )

    plt.xticks(ind, cols, fontsize=10, rotation=35)
    plt.xlabel('')
    plt.ylabel('Phenotype Contribution Scores', fontsize=10)
    return fig

pheno_contribs_to_lat(guide_df, interested_traits_guide, topn=5)

In [ ]:
interested_traits_guide = [
                    'GUIDE Lat7', 
                    'GUIDE Lat80', 
                    'GUIDE Lat16', 
                    'GUIDE Lat77', 
                    'GUIDE Lat96', 
                    'GUIDE Lat3',         
]

interested_traits_guide = [
                    'GUIDE Lat7', 
                    'GUIDE Lat96', 
                    'GUIDE Lat3', 
                    'GUIDE Lat80',
                    'GUIDE Lat16', 
                    'GUIDE Lat65',  
                    'GUIDE Lat5', 
                    'GUIDE Lat27', 
]

colors = list(pd.read_csv('./colors.txt', sep='\t', header=None)[0])
middle_ys_cyto, bar_colors = plot_stacked_bar_per_pheno_detailed_cyto(
                                         cyto_contrib_guide, 
                                         interested_traits_guide, spacing=3.5,
                                         path_to_rel_df=paths,
                                         figname='combined_contribs_cyto.png', 
                                         topn=20, 
                                         topn_genes=1,
                                         colors=None, 
                                         gene2cyto_dict=gene2cyto_dict_guide)

In [ ]:
plt.scatter([i for i in range(100)], [i for i in range(100)])
plt.figure(figsize=(20, 10), dpi=200)
plt.xlabel('FEV1 Best Measure\nNumber of Cigarettes Currently Smoking Daily\nSitting Height\nHand Grip Strength (right)\nSORT1', horizontalalignment='right')
plt.savefig('large_letters.png', dpi=200,)

In [ ]:
contribution_phe = gz['contribution_phe']
print(contribution_phe.shape)

In [ ]:
contrib_df = pd.DataFrame(dict(zip(phenos, contribution_phe)))

In [ ]:
def get_safe_phe_name(phe):
    return ''.join([c if (c.isalnum() or c in ['_', '.'] ) else '_' for c in phe])    


**generating df for guide browser (https://guide-analysis.hail.is/)**

In [ ]:
def get_top_var_components(phename):
    p = get_safe_phe_name(phename)
    df = pd.read_csv(f'./all_phenos/phenotypes/guide/{p}/squared_cosine_scores.tsv', sep='\t')
    return str(list(df['PC_(zero_based)'][:5])).strip('[]')

def get_top_n_var_components(phename, n):
    p = get_safe_phe_name(phename)
    df = pd.read_csv(f'./all_phenos/phenotypes/guide/{p}/squared_cosine_scores.tsv', sep='\t')
    return list(df['squared_cosine_score'][:n])

alz = sum(get_top_n_var_components("Alzheimer's disease/dementia", 5))* 100

In [ ]:
dd = dict()
for p in phenos:
    dd[p] = [get_top_var_components(p)]
    for i in [1, 3, 5]:
        dd[p].append(f'{sum(get_top_n_var_components(p, i))* 100:.2f}%')

new_df = pd.DataFrame(dd)
new_df.index=['Top 5 Latent Factor Indices (ordered by variance component)', 
              'Top 1 Variance Component', 
              'Top 3 Variance Component', 
              'Top 5 Variance Component']
new_df = new_df.T
new_df.to_csv('../browser/pheno_table.tsv', sep='\t')
pheno_table = pd.read_csv('../browser/pheno_table.tsv', sep='\t').T
pheno_table.rename({'Unnamed: 0' : 'Phenotype'})
pheno_table.columns = pheno_table
new_df = new_df.reset_index(names = 'Phenotype')
new_df.to_csv('../browser/pheno_table.tsv', sep='\t')